# COVID-19 Analysis: A Deeper Dive into the Stats
## ( + easy interactive figures with plot.ly) 

3/9/20

According to the "Coronavirus COVID-19 Global Cases by Johns Hopkins CSSE" dashboard, as of right now, there are a total of 113584 confirmed cases, 3996 deaths, and 62496 recovered. From the media reports, it feels like outbreaks are happening at an exponential rate. However, hearing these numbers being thrown everywhere and used to both support or refute the need to panic, I decided to dive deeper into the numbers myself. How reliable are these reports? What do they really say about the threat of the virus? I don't know...
   

Some initial notes/thoughts/findings from browsing the internet:
    - Confirmed cases include presumptive cases
    - Confirmed cases are laboratory-confirmed using PCR 
        - (sidenote: with what experience I've had with QT-PCR, results can be finicky and may vary significantly if proper mixing and sampling isn't done) 
        - According to the WHO daily situation reports, a confirmed case is "A person with laboratory confirmation of COVID-19 infection, irrespective of clinical signs and symptoms."
        - There is no single protocol.
            - Following the link to the laboratory testing page, there are several different protocols coming from several different countries including the US, China, Thailand, etc. 
            - The primers/probe combinations used for different protocols are different. The targets are different... 
        - Recovered patients who have consecutive negative test results test positive after an additional quarentine period?! ["Positive RT-PCR Test Results in Patients Recovered From COVID-19"](https://jamanetwork.com/journals/jama/fullarticle/2762452)
<a href="url" target="_blank">hyperlinked words</a>

In [1]:
import numpy as np
import pandas as pd
import scipy as sp

# import plotly.offline as py
# py.init_notebook_mode(connected=True)



import plotly.graph_objects as go 
import plotly.figure_factory as ff
import plotly.express as px
import plotly.io as pio  

pd.set_option("display.min_rows", 15)


pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)

In [2]:
import plotly.io as pio
pio.renderers
pio.renderers.default = 'notebook'
%load_ext autoreload
%autoreload 2

In [3]:
"""
# reload all changed modules before executing a new line
%load_ext autoreload
%autoreload 2

# save figures as static images
fig = go.FigureWidget(data=go.Bar(y=[2, 3, 1]))
fig.write_image('figure.png')
"""

"\n# reload all changed modules before executing a new line\n%load_ext autoreload\n%autoreload 2\n\n# save figures as static images\nfig = go.FigureWidget(data=go.Bar(y=[2, 3, 1]))\nfig.write_image('figure.png')\n"

#### Mapping out the Deaths

Out of all the stats, I would say the number of deaths can be "trusted" most (ie if someone is said to have died from the virus, it is highly probable that they had been infected).


In [4]:
df = pd.read_csv('csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')

In [5]:
total_deaths=df[df.columns[-1]].sum()
print("total deaths as of {} : {} ".format(df.columns[-1],total_deaths))

total deaths as of 3/11/20 : 4647 


In [6]:
# Check that deaths are only increasing
not_monotonic=[]
dft=df.T[4:]
for col in dft.columns:
    monotonic=dft[col].is_monotonic_increasing
    if monotonic==False:
        print(states[int(col)],  ": INCONSISTANT")
        not_monotonic.append([int(col)])
print("Inconsistancies: {}".format(len(not_monotonic)))

# fill in countries
df['Country/Region']=df['Country/Region'].fillna(method="ffill")
df['Province/State']=df['Province/State'].fillna(value=df['Country/Region'])
print(df)

Inconsistancies: 0
       Province/State Country/Region      Lat      Long  1/22/20  1/23/20  \
0            Thailand       Thailand  15.0000  101.0000        0        0   
1               Japan          Japan  36.0000  138.0000        0        0   
2           Singapore      Singapore   1.2833  103.8333        0        0   
3               Nepal          Nepal  28.1667   84.2500        0        0   
4            Malaysia       Malaysia   2.5000  112.5000        0        0   
5    British Columbia         Canada  49.2827 -123.1207        0        0   
6     New South Wales      Australia -33.8688  151.2093        0        0   
..                ...            ...      ...       ...      ...      ...   
397     Minnehaha, SD             US  43.6632  -96.8351        0        0   
398     Bon Homme, SD             US  42.9815  -97.8722        0        0   
399       Socorro, NM             US  33.8837 -106.7235        0        0   
400    Bernalillo, NM             US  35.0178 -106.6291  

In [7]:
# df['bin_lat']=pd.cut(df['Lat'], bins=18)
# df['bin_long']=pd.cut(df['Long'], bins=18)
df['bin_lat'],blat=pd.cut(df['Lat'], bins=np.linspace(-180, 180, 10), precision=0,retbins=True)
df['bin_long'],blong=pd.cut(df['Long'], bins=np.linspace(-180, 180, 18),precision=0,retbins=True)

df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,bin_lat,bin_long
0,Thailand,Thailand,15.0000,101.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,"(-20.0, 20.0]","(95.0, 116.0]"
1,Japan,Japan,36.0000,138.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,2,4,4,5,6,6,6,6,6,6,6,6,10,10,15,"(20.0, 60.0]","(138.0, 159.0]"
2,Singapore,Singapore,1.2833,103.8333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(-20.0, 20.0]","(95.0, 116.0]"
3,Nepal,Nepal,28.1667,84.2500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(20.0, 60.0]","(74.0, 95.0]"
4,Malaysia,Malaysia,2.5000,112.5000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(-20.0, 20.0]","(95.0, 116.0]"


In [8]:
#centers=[int(np.average([blat[i],blat[i-1]])) for i in np.arange(1,18)]

In [9]:
initial=df.groupby(['bin_lat','bin_long'])['3/8/20'].sum().sort_values(ascending=False).dropna().reset_index() #.fillna(0)
print(initial.head(20))

initial_total=df['3/8/20'].sum()
print('total deaths as of 3/8/20 (start of analysis): ', initial_total)

           bin_lat          bin_long  3/8/20
0     (20.0, 60.0]     (95.0, 116.0]  3056.0
1     (20.0, 60.0]      (11.0, 32.0]   368.0
2     (20.0, 60.0]      (53.0, 74.0]   194.0
3     (20.0, 60.0]    (116.0, 138.0]    86.0
4     (20.0, 60.0]     (-11.0, 11.0]    44.0
5     (20.0, 60.0]  (-138.0, -116.0]    19.0
6     (20.0, 60.0]    (138.0, 159.0]    12.0
7    (-20.0, 20.0]     (95.0, 116.0]     7.0
8     (20.0, 60.0]      (32.0, 53.0]     6.0
9     (20.0, 60.0]      (74.0, 95.0]     3.0
10    (20.0, 60.0]    (-95.0, -74.0]     2.0
11  (-60.0, -20.0]    (138.0, 159.0]     2.0
12   (-20.0, 20.0]    (116.0, 138.0]     1.0
13  (-60.0, -20.0]    (-74.0, -53.0]     1.0
14  (-60.0, -20.0]     (95.0, 116.0]     1.0
15   (-20.0, 20.0]    (-32.0, -11.0]     0.0
16  (-60.0, -20.0]    (159.0, 180.0]     0.0
17  (-60.0, -20.0]      (53.0, 74.0]     0.0
18  (-60.0, -20.0]      (11.0, 32.0]     0.0
19   (-20.0, 20.0]    (-95.0, -74.0]     0.0
total deaths as of 3/8/20 (start of analysis):  3802


In [10]:
today=df.columns[-3]

latest=df.groupby(['bin_lat','bin_long'])[today].sum().sort_values(ascending=False).dropna().reset_index() #.fillna(0)
print(latest.head())
print(latest.tail())

print('\n\nRemoving 0s')
latest=latest[latest[today]>0]
print(latest.head())
print(latest.tail())

        bin_lat        bin_long  3/11/20
0  (20.0, 60.0]   (95.0, 116.0]   3117.0
1  (20.0, 60.0]    (11.0, 32.0]    833.0
2  (20.0, 60.0]    (53.0, 74.0]    354.0
3  (20.0, 60.0]   (-11.0, 11.0]    127.0
4  (20.0, 60.0]  (116.0, 138.0]     96.0
           bin_lat          bin_long  3/11/20
28   (60.0, 100.0]  (-159.0, -138.0]      0.0
29   (60.0, 100.0]    (-32.0, -11.0]      0.0
30  (-60.0, -20.0]      (53.0, 74.0]      0.0
31  (-60.0, -20.0]      (11.0, 32.0]      0.0
32    (20.0, 60.0]  (-159.0, -138.0]      0.0


Removing 0s
        bin_lat        bin_long  3/11/20
0  (20.0, 60.0]   (95.0, 116.0]   3117.0
1  (20.0, 60.0]    (11.0, 32.0]    833.0
2  (20.0, 60.0]    (53.0, 74.0]    354.0
3  (20.0, 60.0]   (-11.0, 11.0]    127.0
4  (20.0, 60.0]  (116.0, 138.0]     96.0
           bin_lat        bin_long  3/11/20
14   (-20.0, 20.0]  (-95.0, -74.0]      1.0
15  (-60.0, -20.0]   (95.0, 116.0]      1.0
16   (60.0, 100.0]    (11.0, 32.0]      1.0
17  (-60.0, -20.0]  (-74.0, -53.0]      1.

In [11]:

countries=list(df.groupby('Country/Region').groups.keys())
print(countries)


['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Cambodia', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Congo (Kinshasa)', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cruise Ship', 'Cyprus', 'Czechia', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Estonia', 'Finland', 'France', 'French Guiana', 'Georgia', 'Germany', 'Greece', 'Holy See', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Korea, South', 'Kuwait', 'Latvia', 'Lebanon', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Malaysia', 'Maldives', 'Malta', 'Martinique', 'Mexico', 'Moldova', 'Monaco', 'Mongolia', 'Morocco', 'Nepal', 'Netherlands', 'New Zealand', 'Nigeria', 'North Macedonia', 'Norway', 'Oman', 'Pakistan', 'Panama', 'Parag

In [12]:
# Group cumulative stats by lat/long coordinates since the number of deaths by province/state is sparse

latest['long']=latest['bin_long'].map({i:i.mid for i in latest['bin_long']})

latest['lat']=latest['bin_lat'].map({i:i.mid for i in latest['bin_lat']})

print(latest.head())
max_country=df[df[today]==df[today].max()]['Country/Region'].values[0]
max_province=df.iloc[df[df[today]==df[today].max()]['Country/Region'].index]['Province/State'].values[0]
print('\n Maximum death toll: {} in {}, {}\n'.format(df[today].max(),max_province, max_country))
total_deaths=df[today].sum()
print('Total deaths: ', total_deaths)

        bin_lat        bin_long  3/11/20   long   lat
0  (20.0, 60.0]   (95.0, 116.0]   3117.0  105.5  40.0
1  (20.0, 60.0]    (11.0, 32.0]    833.0   21.5  40.0
2  (20.0, 60.0]    (53.0, 74.0]    354.0   63.5  40.0
3  (20.0, 60.0]   (-11.0, 11.0]    127.0    0.0  40.0
4  (20.0, 60.0]  (116.0, 138.0]     96.0  127.0  40.0

 Maximum death toll: 3046 in Hubei, China

Total deaths:  4647


In [13]:
states=df['Province/State'].values

In [25]:
# Global density plot of Deaths
latest['scaled_deaths']=np.log(latest[today])**2

fig = px.density_mapbox(latest, lat='lat', lon='long', z='scaled_deaths', title="Map of Death Counts", radius=25, height=600,
                        center=dict(lat=30, lon=110), zoom=1,
                        mapbox_style="carto-positron")
fig

In [15]:
# fig_mod = go.Figure(fig)

# max_deaths=max(latest['scaled_deaths'])
# fig_mod.update_layout(hovertext='today')
# fig_mod.show()

In [16]:
# #scatter
# import math
# hover_text = []
# bubble_size = []

# for index, row in df.iterrows():
#     hover_text.append(('Country/Region: {country}<br>'+
#                       'Date: {date}<br>'+
#                       'Number of Deaths: {death}<br>').format(country=df["Country/Region"],
#                                             date=today,
#                                             death=row[today]))
#     bubble_size.append(math.sqrt(row[today]))

# df['text'] = hover_text
# df['size'] = bubble_size
# sizeref = 2.*max(df['size'])/(100**2)

print("Scatter Plot of Countries/Regions where Deaths>0 as of {}".format(today))
# fig_scat = px.scatter_mapbox(df, lat="Lat", lon="Long", hover_name="Country/Region" ,hover_data=["Long","Lat", today], color_discrete_sequence=["fuchsia"], zoom=1, height=300)
fig_scat = px.scatter_mapbox(df[df[today]>0], lat="Lat", lon="Long", hover_name="Country/Region" ,hover_data=["Long","Lat", 'Province/State',today],color_discrete_sequence=["fuchsia"],zoom=1, height=600)
fig_scat.update_layout(mapbox_style="open-street-map")
fig_scat  #.show()

# fig_mod = go.Figure(fig_scat)
# fig_mod.update_layout(hovertext='today')


Scatter Plot of Countries/Regions where Deaths>0 as of 3/11/20


#### Alternative Mapbox Styles (raster tiles)
- maps that do not require an API token: 
    - `mapbox_style`=`"open-street-map"`, `"carto-positron"`, `"carto-darkmatter"`, `"stamen-terrain"`, `"stamen-toner"`, or `"stamen-watercolor" `
    - Base Tiles from the USGS: 
    ```
    fig.update_layout(
        mapbox_style="white-bg",
        mapbox_layers=[
            {
                "below": 'traces',
                "sourcetype": "raster",
                "source": [
                    "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
                ]
            }
          ])```
- maps that require a signup or token: "basic", "streets", "outdoors", "light", "dark", "satellite", or "satellite-streets"
    - Base Tiles from the USGS, radar overlay from Environment Canada: no token needed:
    ``` 
    fig.update_layout(
        mapbox_style="white-bg",
        mapbox_layers=[
            {
                "below": 'traces',
                "sourcetype": "raster",
                "source": [
                    "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
                ]
            },
            {
                "sourcetype": "raster",
                "source": ["https://geo.weather.gc.ca/geomet/?"
                           "SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&BBOX={bbox-epsg-3857}&CRS=EPSG:3857"
                           "&WIDTH=1000&HEIGHT=1000&LAYERS=RADAR_1KM_RDBR&TILED=true&FORMAT=image/png"],
            }
          ]) ```
    - to provide token, set `layout.mapbox.access_token` (or, if using Plotly Express, via the `px.set_mapbox_access_token()` configuration function)
- 

##### Generally, if your layout.mapbox.style does not use Mapbox service data, you do not need to register for a Mapbox account.



In [17]:
fig.add_trace(fig_scat.data[0])

# fig.write_html("mapbox_scatter-density_plot_deaths.html")


fig

# uncomment to save figure
#fig_scat.write_html("mapbox_scatter_plot_deaths.html")


In [18]:
# png = go.FigureWidget(data=fig)
# png.write_image('mapbox_scatter_plot_deaths.png')

In [19]:
#compare to confirmed

dfc = pd.read_csv('csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
dfc['Country/Region']=dfc['Country/Region'].fillna(method="ffill")
# Check that deaths are only increasing
not_monotonic=[]
dfct=dfc.T[4:]
for col in dfct.columns:
    monotonic=dft[col].is_monotonic_increasing
    if monotonic==False:
        print(states[int(col)],  ": INCONSISTANT")
        not_monotonic.append([int(col)])
print("Inconsistancies: {}".format(len(not_monotonic)))



Inconsistancies: 0


In [20]:
figc = px.density_mapbox(dfc, lat='Lat', lon='Long', z=today, radius=50,
                        center=dict(lat=30, lon=110), zoom=1,
                        mapbox_style="carto-positron")
figc

In [21]:
print("Scatter Plot of Confirmed Cases as of {}".format('3/8/20'))
# fig_scat = px.scatter_mapbox(df, lat="Lat", lon="Long", hover_name="Country/Region" ,hover_data=["Long","Lat", today], color_discrete_sequence=["fuchsia"], zoom=1, height=300)
ifigc_scat = px.scatter_mapbox(dfc[dfc['3/8/20']>0], lat="Lat", lon="Long", hover_name="Country/Region" ,hover_data=["Long","Lat", 'Province/State','3/8/20'],color_discrete_sequence=["fuchsia"],zoom=1, height=600)
ifigc_scat.update_layout(mapbox_style="open-street-map")
ifigc_scat

Scatter Plot of Confirmed Cases as of 3/8/20


In [22]:
print("Scatter Plot of Confirmed Cases as of {}".format(today))
# fig_scat = px.scatter_mapbox(df, lat="Lat", lon="Long", hover_name="Country/Region" ,hover_data=["Long","Lat", today], color_discrete_sequence=["fuchsia"], zoom=1, height=300)
figc_scat = px.scatter_mapbox(dfc[dfc[today]>0], lat="Lat", lon="Long", hover_name="Country/Region" ,hover_data=["Long","Lat", 'Province/State',today],color_discrete_sequence=["fuchsia"],zoom=1, height=600)
figc_scat.update_layout(mapbox_style="open-street-map")
figc_scat



Scatter Plot of Confirmed Cases as of 3/11/20


In [23]:
figc.add_trace(ifigc_scat.data[0])
# figc.write_html("mapbox_scatter-density_plot_confirmed.html")
figc


In [24]:
#troubleshooting 
#print(vars(figc))
print(figc.layout.coloraxis)

layout.Coloraxis({
    'colorbar': {'title': {'text': '3/11/20'}},
    'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                   [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                   '#9c179e'], [0.4444444444444444, '#bd3786'],
                   [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                   '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                   [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']]
})
